# 类型系统 

+ 与二进制存储结构相关联的成为元类型/原语类型： `primitive type`, 如`Int32`, `Int64`这种，在定义元类型时要声明**位数**， 且位数必须为8的倍数
+ 类型名第一个字母大写，包括自定义的类型、结构体等

## 元类型

In [40]:
# 256位正数
primitive type Int256 <: Integer 256 end

In [41]:
# 8位浮点数
primitive type Float8 <: AbstractFloat 8 end

与抽象概念相关，被用来继承的类型为`Abstract Type`, 如`Real`, `Integer`, `AbstractFloat`

## 抽象类型

In [42]:
abstract type name end
abstract type family_name <: name end

In [43]:
family_name <: name

true

In [44]:
supertype(family_name)

name

In [45]:
subtypes(name)

1-element Vector{Any}:
 family_name

In [46]:
# 并不会递归查询
subtypes(Integer)

4-element Vector{Any}:
 Bool
 Int256
 Signed
 Unsigned

##  类型的类型

In [47]:
typeof(Int64)

DataType

In [48]:
typeof(DataType)

DataType

所有类型的类型是`DataType`，所有类型的父类型是`Any`, 所有类型的子类型都包含`Union`

In [49]:
Real <: Any, typeof(Union) <: DataType

(true, true)

In [50]:
# DataType和Any都是类型，Any是所有类型的父类型，而DataType是所有类型的类型
DataType <: Any, typeof(Any) <: DataType

(true, true)

In [51]:
supertype(Union)

Type{T}

`DataType`的父类型是参数化类型`Type{T}`

In [52]:
supertype(DataType)

Type{T}

`Type{T}`可以看作类型选择器

In [53]:
isa(Int64, Type{Int64})

true

## 类型选择器Type Selector `::Type{T}`

In [54]:
# Type{T}的使用
# 第一个参数为non-usable parameter, 意思是在函数体中无法使用（因为他没有名字），
function convert_type(::Type{T}, x::Integer) where T <: AbstractFloat
    return convert(T, x)
end

convert_type (generic function with 1 method)

In [55]:
# 调用该函数，第一个参数为类型
convert_type(Float64, 1)

1.0

In [56]:
# 区分一下：：T
# 第一个参数也为non-usable parameter, 意思是在函数体中无法使用（因为他没有名字）
f(::T, x) where T = convert(T, x)

f (generic function with 1 method)

In [57]:
# 第一个参数是值，该函数将第二个参数的类型转换为第一个参数的类型
f(1.0, 2)

2.0

##  复合类型

In [58]:
abstract type Foo end

In [59]:
struct FooA <: Foo
    a::Any
    b::Float64
end

In [60]:
dump(FooA)

FooA <: Foo
  a::Any
  b::Float64


In [61]:
FooA <: Foo, fieldnames(FooA)

(true, (:a, :b))

如何实例化一个结构体？

+ 按照struct中类型指定，手动指定每一个field的类型
+ 不指定每一个field的类型，julia会自动进行convert到struct的类型指定（不建议）

> 一个struct实例化后，如果更改某一个field的值，他会自动将该值convert到struct的类型指定。

任何一个struct在定义时对field进行了类型声明，那么任何一个实例化的struct的field的值都是该field的类型

In [62]:
fooa = FooA("xue", 23)

FooA("xue", 23.0)

In [63]:
dump(fooa)

FooA
  a: String "xue"
  b: Float64 23.0


## Callable Struct(闭包、closure)

函数内定义函数(好像也是闭包)， Julia很难进行优化
```Julia
funtion f1()
    function f2()
    end
end
```

In [64]:
struct abb
    a::Int64
    b::Float64
end

In [65]:
# 目的是利用构建的struct创建一个新函数
function (f::abb)(c)
    return f.a + f.b + c
end

In [66]:
abb_f = abb(1, 2)

abb(1, 2.0)

In [67]:
methods(abb_f)

# 1 method for callable object:
 [1] (f::abb)(c)
     @ In[65]:2

In [68]:
abb_f(1)

4.0

## 参数化类型

### 参数化复合类型

In [69]:
mutable struct Point1D{T <: Real}
    x::T
end

In [70]:
# Poin1D可以被看作一种简写
Point1D{<:Real} == Point1D

true

In [71]:
# 即使参数存在继承关系, 参数化符合类型也不存在继承关系
# 专业术语叫作“抗变”
# 抗变只发生在container类型中，标量没有这个特性
Point1D{Int64} <: Point1D{Integer}

false

Julia自带的数据类型像是`Dict`，`Vector`，`Matrix`也可以看成结构体一样的东西（container），因此也遵循抗变的特征

在Julia定义函数时（详见[函数参数类型标注](https://docs.julialang.org/en/v1/manual/functions/#Argument-type-declarations)），使用`：：`对参数进行类型声明
+ 如`f(x::Real) = x + 1`，那么`x`只要是`Real`的任意子类型，该函数都能被调用。
+ 如果参数是以container的形式出现，务必要注意抗变的特性。
    + 例如函数`f(x::AbstractVector{Real})`，输入只要是AbstractVector的子类型就可以，但是这个container内的元素类型必须也是Real， 因为`AbstractVector{Float64}`并不是`AbstractVector{Real}`的子类型。最好的办法是声明`x::AbstractVector{<:Real}`或者`f(x::AbstractVector{T}) where {T <: Real}`
    + 如果是更加复杂的嵌套container，更要注意抗变的特性，尤其是和参数类型结合在一起的时候。例如`f(x::Dict{Int, Vector{<:Real}})`，当输入`Dict(1 => [1, 2.0], 2 => [1, 3.0])`时找不到函数去执行，原因在于抗变的特性，虽然container内的元素是子类型，但是container不是子类型

In [151]:
function f1(x::Dict{Int, <:AbstractVector{<:Real}})
    rst = 0.0
    for (k, v) in x
        rst += sum(v)
    end
    return rst
end

f1 (generic function with 3 methods)

In [153]:
@which f1(Dict(1 => [1, 2.0], 2 => [1, 3.0]))

f1(x::Dict{Int64, <:AbstractVector{<:Real}})
     @ Main In[151]:1

In [154]:
function f2(x::Dict{Int, <:Vector{<:Real}})
    rst = 0.0
    for (k, v) in x
        rst += sum(v)
    end
    return rst
end

f2 (generic function with 1 method)

In [155]:
@which f2(Dict(1 => [1, 2.0], 2 => [1, 3.0]))

f2(x::Dict{Int64, <:Vector{<:Real}})
     @ Main In[154]:1

In [156]:
typeof(Dict(1 => [1, 2.0], 2 => [1, 3.0])) <: Dict{Int64, <:Vector{<:Real}}

true

In [162]:
Dict{Int64, <:Vector{<:Real}} <: Dict{Int, <:AbstractVector{<:Real}}

true

In [166]:
isa(Dict(1 => [1, 2.0], 2 => [1, 3.0]), Dict{Int64, <:Vector{<:Real}})

true

In [167]:
isa(Dict(1 => [1, 2.0], 2 => [1, 3.0]), Dict{Int64, <:AbstractVector{<:Real}})

true

In [168]:
isa(Dict(1 => 1.0:2.0, 2 => [1, 3.0]), Dict{Int64, <:AbstractVector{<:Real}})

true

In [172]:
@code_warntype Dict(1 => 1.0:2.0, 2 => 1.0:3.0)

MethodInstance for Dict(::Pair{Int64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, ::Pair{Int64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}})
  from Dict(ps::Pair{K, V}...) where {K, V} @ Base dict.jl:114
Static Parameters
  K = Int64
  V = StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}
Arguments
  #self#::Type{Dict}
  ps::Tuple{Pair{Int64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, Pair{Int64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}}
Body::Dict{Int64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}
1 ─ %1 = $(Expr(:static_parameter, 1))::Core.Const(Int64)
│   %2 = $(Expr(:static_parameter, 2))::Core.Const(StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64})
│   %3 = Core.apply_ty

### 参数化元类型

In [78]:
primitive type ptr{T} 64 end

### 参数化基本原理（跳过）

从这一块到最后有点麻烦，先跳过

### 参数化继承(跳过）

## 常用数据类型

### 元组

In [79]:
tp1 = (1.0, 2.0, 3.0, 4.0)

(1.0, 2.0, 3.0, 4.0)

In [80]:
typeof(tp1)

NTuple{4, Float64}

In [135]:
# NTuple不是一个复合类型
# tp3 = NTuple{3, Float64}(1.0, 2.0, 3.0)

In [136]:
tp1[1]

1.0

In [137]:
tp2 = (1, 2, (3, 4, 5)..., 6, 7)

(1, 2, 3, 4, 5, 6, 7)

In [138]:
tp3 = (1, "a", "china")

(1, "a", "china")

### 命名元组🚀🚀🚀

In [139]:
ntp1 = (a = 1, b = 2, c = 3)

(a = 1, b = 2, c = 3)

In [140]:
a = 1
b = 2
c = 3
(; a, b, c)

(a = 1, b = 2, c = 3)

In [141]:
NamedTuple{(:a, :b, :c)}(ntuple(i -> i ^ 3, Val(3)))

(a = 1, b = 8, c = 27)

In [142]:
ntuple(Val(3)) do i 
    i ^ 2
end

(1, 4, 9)

In [143]:
typeof(ntp1)

@NamedTuple{a::Int64, b::Int64, c::Int64}

In [144]:
# 四种索引的方式
ntp1.a == ntp1[1] == getfield(ntp1, :a) == getindex(ntp1, :a)

true

In [145]:
# 说明命名元组是一个参数化复合类型
ntp2 = NamedTuple{(:a, :b, :c),Tuple{Int64,Int64,Float64}}((1, 2, 3.0))

(a = 1, b = 2, c = 3.0)

In [146]:
# iter over a ntuple, show its value by default
for i in ntp1
    println(i)
end

1
2
3


In [93]:
# to iter through each key and value, use pairs
for (i, j) in pairs(ntp1)
    println(i, " => ", j)
end

a => 1
b => 2
c => 3


### 键值对

In [94]:
dump(Pair) # 先只关注body部分吧，其它的部分先别管

UnionAll
  var: TypeVar
    name: Symbol A
    lb: Union{}
    ub: Any
  body: UnionAll
    var: TypeVar
      name: Symbol B
      lb: Union{}
      ub: Any
    body: Pair{A, B} <: Any
      first::A
      second::B


In [95]:
fieldnames(Pair)

(:first, :second)

In [96]:
p1 = Pair(1, 3.2)

1 => 3.2

In [97]:
typeof(p1)

Pair{Int64, Float64}

In [98]:
p2 = 2 => "3"

2 => "3"

In [99]:
p1.first

1

In [100]:
p2.second

"3"

In [101]:
# Pair to Turple
Tuple(p1)

(1, 3.2)

### 字典

In [102]:
d1 = Dict(1 => 1.1, 2 => 2.2)

Dict{Int64, Float64} with 2 entries:
  2 => 2.2
  1 => 1.1

In [103]:
d2 = Dict{String,Int64}("a" => 3, "b" => 4.0)

Dict{String, Int64} with 2 entries:
  "b" => 4
  "a" => 3

In [104]:
# 字典推导式
d3 = Dict(i => i^2 for i ∈ 1:10)

Dict{Int64, Int64} with 10 entries:
  5  => 25
  4  => 16
  6  => 36
  7  => 49
  2  => 4
  10 => 100
  9  => 81
  8  => 64
  3  => 9
  1  => 1

In [105]:
ntp2

(a = 1, b = 2, c = 3.0)

In [106]:
pairs(ntp2)

pairs(::NamedTuple) with 3 entries:
  :a => 1
  :b => 2
  :c => 3.0

In [107]:
# NamedTuple to Dict
d4 = Dict(pairs(ntp2))

Dict{Symbol, Real} with 3 entries:
  :a => 1
  :b => 2
  :c => 3.0

In [108]:
d4[:b]

2

In [109]:
delete!(d4, :a)

Dict{Symbol, Real} with 2 entries:
  :b => 2
  :c => 3.0

In [110]:
pop!(d4, :b)

2

In [111]:
d4

Dict{Symbol, Real} with 1 entry:
  :c => 3.0

In [112]:
d4.count

1

In [113]:
keys(d2)

KeySet for a Dict{String, Int64} with 2 entries. Keys:
  "b"
  "a"

In [114]:
for key in keys(d2)
    println("$key \t", d2[key])
end

b 	4
a 	3


In [115]:
values(d2)

ValueIterator for a Dict{String, Int64} with 2 entries. Values:
  4
  3

In [116]:
# collect iterator
collect(values(d2))

2-element Vector{Int64}:
 4
 3

In [117]:
# Set(可迭代数据结构)
a = Set(1:5)

Set{Int64} with 5 elements:
  5
  4
  2
  3
  1

In [118]:
b = Set([1, 2, 3])

Set{Int64} with 3 elements:
  2
  3
  1

In [119]:
b = Set(("a", 1, 2))

Set{Any} with 3 elements:
  2
  "a"
  1

In [120]:
1 in b

true

In [121]:
isempty(a)

false

In [122]:
empty!(a)

Set{Int64}()

### 缺失值missing 

In [123]:
# NaN是一个数并不代表缺失值

In [124]:
missing + 1

missing

In [125]:
ismissing([1, 2, 3, missing])

false

In [126]:
ismissing.([1, 2, 3, missing])

4-element BitVector:
 0
 0
 0
 1

In [127]:
isless(missing, Inf)

false

### nothing不需要在内存中表达

## 其它数据类型

Julia有一个`DataStructures.jl`， 这个包内置了很多Julia本身没有的数据结构

### `OrderedDict`

In [128]:
using DataStructures

In [129]:
# 有序字典
d1 = OrderedDict('a' => 1, 'b' => 2, 'c' => 3)

OrderedDict{Char, Int64} with 3 entries:
  'a' => 1
  'b' => 2
  'c' => 3

In [130]:
# 无序字典（Julia内置）
d2 = Dict('a' => 1, 'b' => 2, 'c' => 3)

Dict{Char, Int64} with 3 entries:
  'a' => 1
  'c' => 3
  'b' => 2

In [131]:
?methodswith

search: methodswith endswith methods mergewith startswith



```
methodswith(typ[, module or function]; supertypes::Bool=false])
```

Return an array of methods with an argument of type `typ`.

The optional second argument restricts the search to a particular module or function (the default is all top-level modules).

If keyword `supertypes` is `true`, also return arguments with a parent type of `typ`, excluding type `Any`.

See also: [`methods`](@ref).


`methodswith`函数可以查看所有以某个数据结构为参数的所有函数， 例如`methodswith(OrderedDict)`可以查看所有适用于`OrderedDict`的函数， 可以看出包的作者给`OrderedDict`适配了很多的方法

In [132]:
methodswith(OrderedDict)

[1] copy(d::OrderedDict) @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:55
[2] delete!(h::OrderedDict, key) @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:448
[3] empty(d::OrderedDict, ::Type{K}, ::Type{V}) where {K, V} @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:89
[4] empty(d::OrderedDict{K, V}) where {K, V} @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:88
[5] empty!(h::OrderedDict{K, V}) where {K, V} @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:229
[6] get(h::OrderedDict{K, V}, key, default) where {K, V} @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:390
[7] get(default::Union{Function, Type}, h::OrderedDict{K, V}, key) where {K, V} @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:395
[8] get!(h::OrderedDict{K, V}, key0, default) where {K, V} @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:346
[9] get!(default::Union{Function, Type}, h::OrderedDict{K, V}, key0) where {K, V} @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:361
[10] getindex(h::OrderedDict{K, V}, key) where {K, V} @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:385
[11] getkey(h::OrderedDict{K, V}, key, default) where {K, V} @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:403
[12] haskey(h::OrderedDict, key) @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:400
[13] isempty(d::OrderedDict) @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:92
[14] iterate(t::OrderedDict) @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:454
[15] iterate(t::OrderedDict, i) @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:459
[16] last(h::OrderedDict) @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:515
[17] length(d::OrderedDict) @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:91
[18] merge(d::OrderedDict, others::AbstractDict...) @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:488
[19] merge(combine::Function, d::OrderedDict, others::AbstractDict...) @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:498
[20] mergewith(combine, d::OrderedDict, others::AbstractDict...) @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:493
[21] pop!(h::OrderedDict) @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:414
[22] pop!(h::OrderedDict, key) @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:428
[23] pop!(h::OrderedDict, key, default) @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:433
[24] popfirst!(h::OrderedDict) @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:421
[25] setindex!(h::OrderedDict{K, V}, v0, key0) where {K, V} @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:327
[26] similar(d::OrderedDict) @ OrderedCollections deprecated.jl:103
[27] sizehint!(d::OrderedDict, newsz) @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/ordered_dict.jl:215
[28] sort!(d::OrderedDict; byvalue, args...) @ OrderedCollections ~/.julia/packages/OrderedCollections/Xihhq/src/dict_sorting.jl:4

In [133]:
zipit = zip(1:5, 'a':'e')

zip(1:5, 'a':1:'e')

In [134]:
OrderedDict(zipit)

OrderedDict{Int64, Char} with 5 entries:
  1 => 'a'
  2 => 'b'
  3 => 'c'
  4 => 'd'
  5 => 'e'